In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
import scrapy
import random
import time
from urllib.parse import urljoin
import re

In [2]:
arquivos_excel = ['sites_36.xlsx']
dados_brutos = [pd.read_excel(i) for i in arquivos_excel]
base = pd.concat(dados_brutos, ignore_index=True)

In [13]:
base.head(2)

,id,name,url,state,city,"sitemap_posts_url (sem data, index, webstories)",Google News Sitemap URL,rss_url (se presente),paywall,name_normalized,name_normalized_cortado
0,875,CLICKON,http://www.clickon.com.br/,SP,São Paulo,NaN,NaN,NaN,NaN,clickon,[clickon]
1,876,CLICKSINDICO,https://www.clicksindico.com.br/,RS,Porto Alegre,https://www.clicksindico.com.br/sitemap.xml,NaN,https://www.clicksindico.com.br/blog-feed.xml,NaN,clicksindico,[clicksindico]


In [4]:
# Alteração na base normalizada cortada para a pesquisa no navegador

#base['name_normalized'] = base['name_normalized'].str.replace(r'(estadao|com|\-|folha|g1|uol|\.)', '', regex=True)

In [5]:
# Alteração  na base Normalizada cortado alguns valores

base['name_normalized_cortado'] = base['name_normalized'].str.replace(r'\b(blog|no|na|do|da)\b', '', regex=True)
# Remover espaços extras resultantes da substituição
base['name_normalized_cortado'] = base['name_normalized_cortado'].str.strip().str.replace(r'\s+', ' ', regex=True)
# Supondo que 'base' seja o DataFrame e 'name_normalized_cortado' seja a coluna
base['name_normalized_cortado'] = base['name_normalized_cortado'].apply(lambda x: x.split() if isinstance(x, str) else x)
base['name_normalized_cortado']

0                                  [clickon]
1                             [clicksindico]
2                              [clicrbs, sc]
3                                 [cliktrue]
4                              [clima, surf]
5                               [climatempo]
6                          [clique, esporte]
7                             [club, mobile]
8       [clube, de, criacao, de, sp, (ccsp)]
9                       [clube, vale, serra]
10                 [cmm, comunicacoes, ltda]
11                                   [cmncd]
12                         [cocal, noticias]
13    [coin, gape, -, a, internet, dinheiro]
14                            [colegio, web]
15              [coletivo, magnifica, mundi]
16                         [coluna, d'oeste]
17           [coluna, thais, heredia, -, g1]
18           [coluna, yvonne, maggie, -, g1]
19               [coluna, samy, dana, -, g1]
20                [com, deus, e, a, verdade]
21                       [comando, exercito]
22        

In [7]:
base.iloc[2]['name']

'CLICRBS SC'

## NÃO ESTÁ MAIS FUNCIONANDO😪😪 
## GOOGLE MELHOROU A SEGURANÇA, OU MEU IP ESTÁ BLOQUEADO

In [ ]:


def get_url_from_google(index):
	def ajusta_url(url):
    # Define um padrão para ".com/" ou ".com.br/"
		pattern = r"^(https?://[^/]+?(?:\.com(?:\.br)?/))"
		match = re.match(pattern, url)
		if match:
			return match.group(1)
		else: return url


	query = "+".join(base['name_normalized'].iloc[index].split())
	search_url = f"https://www.google.com/search?q={query}"

	# Configuração de headers para simular um navegador
	user_agents = [
	"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
	"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
	"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0"
	]

	headers = {"User-Agent": random.choice(user_agents)}

	# Fazer a requisição ao Google
	response = requests.get(search_url, headers=headers)

	soup = BeautifulSoup(response.text, "html.parser")
	

	main_data=soup.find_all("a")
	#pattern_inverso = rf'href="(https?://[^"]*{'google'}[^"]*\.(com|org)(?:\.br)?[^"]*)'
	#print(main_data)

	keyword = base["name_normalized_cortado"][index]
	pattern = rf'href="(https://.*{keyword}.*\.com/)"'

	link_primeiro_site_encontrado = []
	for keyword in base['name_normalized_cortado'][index]:
		#print(keyword)
		
		pattern = rf'href="(https?://[^"]*{keyword}[^"]*\.(com|org)(?:\.br)?[^"]*)'
		#pattern = rf'href="(https?://[^"]*{keyword}[^"]*\.(com|org)(?:\.br)?/?)"'
		for i in main_data[:21]:
			print(i)
			match = re.search(pattern, str(i))
			if match:
				#print("URL encontrada:", match.group())
				if match.group() not in link_primeiro_site_encontrado:
					if match.group(1) is not None:
						link_primeiro_site_encontrado.append(ajusta_url(match.group(1)))
						
		if link_primeiro_site_encontrado == [] and len(main_data) > 14:
			for element in [8,15]:
				href = main_data[element].get('href')
				if href and len(href) < 100:
					link_primeiro_site_encontrado.append(ajusta_url(href))
		if link_primeiro_site_encontrado == []:
			for i in main_data[8:21]:
				pattern_1 = rf'href="(https?://[^"]*{keyword}[^"]*[^/]*?)"'
				match = re.search(pattern_1, str(i))
				if match:
						link_primeiro_site_encontrado.append(match.group(1))

		# else:
		# 	href = main_data[8].get('href')
		# 	link_primeiro_site_encontrado.append(ajusta_url(href))
			
	return link_primeiro_site_encontrado


def pega_url_de_lista_com_timesleep(inicio, termino, base):
	fim = dict()
	for index in range(inicio, termino):
		url = get_url_from_google(index)
		#print(f"URL encontrada para '{site}': {url}")
		fim[base.iloc[index]['name']] = url
		# Pausa aleatória entre as requisições
		time.sleep(random.uniform(3, 15))
	
	return fim



In [12]:
get_url_from_google(4)

[]

In [17]:
base.iloc[3]

id                                                                                   878
name                                                                            CLIKTRUE
url                                                           https://www.clicktrue.biz/
state                                                                                 SC
city                                                                       Florianópolis
sitemap_posts_url (sem data, index, webstories)    https://www.clicktrue.biz/sitemap.xml
Google News Sitemap URL                                                              NaN
rss_url (se presente)                                                                NaN
paywall                                                                              NaN
name_normalized                                                                 cliktrue
name_normalized_cortado                                                       [cliktrue]
Name: 3, dtype: objec

In [21]:
c = pega_url_de_lista_com_timesleep(10, 12, base)
c

<a href="/httpservice/retry/enablejs?sei=tGQAaKinHLaL5OUPgOWJsQs">aqui</a>
<a href="/search?q=cmm+comunicacoes+ltda&amp;sca_esv=db757fb3b2a85d69&amp;emsg=SG_REL&amp;sei=tGQAaKinHLaL5OUPgOWJsQs">click here</a>
<a href="https://support.google.com/websearch">feedback</a>
<a href="/httpservice/retry/enablejs?sei=tGQAaKinHLaL5OUPgOWJsQs">aqui</a>
<a href="/search?q=cmm+comunicacoes+ltda&amp;sca_esv=db757fb3b2a85d69&amp;emsg=SG_REL&amp;sei=tGQAaKinHLaL5OUPgOWJsQs">click here</a>
<a href="https://support.google.com/websearch">feedback</a>
<a href="/httpservice/retry/enablejs?sei=tGQAaKinHLaL5OUPgOWJsQs">aqui</a>
<a href="/search?q=cmm+comunicacoes+ltda&amp;sca_esv=db757fb3b2a85d69&amp;emsg=SG_REL&amp;sei=tGQAaKinHLaL5OUPgOWJsQs">click here</a>
<a href="https://support.google.com/websearch">feedback</a>
<a href="/httpservice/retry/enablejs?sei=vWQAaPy7Lry65OUP_-XU0As">aqui</a>
<a href="/search?q=cmncd&amp;sca_esv=db757fb3b2a85d69&amp;emsg=SG_REL&amp;sei=vWQAaPy7Lry65OUP_-XU0As">click here</a>


{'CMM COMUNICACOES LTDA': [], 'CMNCD': []}